In [1]:
import torch
from torchinfo import summary
import s3prl.hub as hub

2022-05-25 00:16:32 | INFO | s3prl.file_utils | PyTorch version 1.11.0 available.


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
model = getattr(hub, "mockingjay")()  # build the Mockingjay model with pre-trained weights
print(model)

Using cache found in /home/chihyuan/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config
UpstreamExpert(
  (transformer): PretrainedTransformer(
    (extracter): OnlinePreprocessor(
      (_melscale): MelScale()
      (_mfcc_trans): MFCC(
        (amplitude_to_DB): AmplitudeToDB()
        (MelSpectrogram): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
      )
    )
    (model): TransformerModel(
      (shared_parameters): ModuleDict()
      (invertible_adapters): ModuleDict()
      (input_representations): TransformerInputRepresentations(
        (spec_transform): Linear(in_features=80, out_features=768, bias=True)
        (LayerNorm): TransformerLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): TransformerEncoder(
        (layer)

In [3]:
summary(model)

Layer (type:depth-idx)                                                 Param #
UpstreamExpert                                                         --
├─PretrainedTransformer: 1-1                                           --
│    └─OnlinePreprocessor: 2-1                                         --
│    │    └─MelScale: 3-1                                              --
│    │    └─MFCC: 3-2                                                  --
│    └─TransformerModel: 2-2                                           --
│    │    └─ModuleDict: 3-3                                            --
│    │    └─ModuleDict: 3-4                                            --
│    │    └─TransformerInputRepresentations: 3-5                       63,744
│    │    └─TransformerEncoder: 3-6                                    85,054,464
│    │    └─PrefixTuningPool: 3-7                                      --
Total params: 85,118,208
Trainable params: 85,118,208
Non-trainable params: 0

In [4]:
type(model.transformer.model)  # the model with adpaters' functions

s3prl.upstream.mockingjay.model.TransformerModel

In [5]:
model.transformer.model.config

TransformerConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "transformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pre_layer_norm": false,
  "share_layer": false,
  "transformers_version": "4.17.0"
}

In [6]:
model.transformer.model.add_adapter("acu", "pfeiffer")
print(model)

2022-05-25 00:20:06 | INFO | s3prl.adapters.configuration | Adding adapter 'acu'.


UpstreamExpert(
  (transformer): PretrainedTransformer(
    (extracter): OnlinePreprocessor(
      (_melscale): MelScale()
      (_mfcc_trans): MFCC(
        (amplitude_to_DB): AmplitudeToDB()
        (MelSpectrogram): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
      )
    )
    (model): TransformerModel(
      (shared_parameters): ModuleDict()
      (invertible_adapters): ModuleDict()
      (input_representations): TransformerInputRepresentations(
        (spec_transform): Linear(in_features=80, out_features=768, bias=True)
        (LayerNorm): TransformerLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): TransformerEncoder(
        (layer): ModuleList(
          (0): TransformerLayer(
            (attention): TransformerAttention(
              (self): TransformerSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_feature

In [7]:
model.transformer.model.train_adapter("acu")
summary(model)

Layer (type:depth-idx)                                                                Param #
UpstreamExpert                                                                        --
├─PretrainedTransformer: 1-1                                                          --
│    └─OnlinePreprocessor: 2-1                                                        --
│    │    └─MelScale: 3-1                                                             --
│    │    └─MFCC: 3-2                                                                 --
│    └─TransformerModel: 2-2                                                          --
│    │    └─ModuleDict: 3-3                                                           --
│    │    └─ModuleDict: 3-4                                                           --
│    │    └─TransformerInputRepresentations: 3-5                                      (63,744)
│    │    └─TransformerEncoder: 3-6                                                   85,948,992
│ 

In [8]:
summary(model, depth=20, row_settings=["depth", "var_names"])

Layer (type (var_name):depth-idx)                                                               Param #
UpstreamExpert                                                                                  --
├─PretrainedTransformer (transformer): 1-1                                                      --
│    └─OnlinePreprocessor (extracter): 2-1                                                      --
│    │    └─MelScale (_melscale): 3-1                                                           --
│    │    └─MFCC (_mfcc_trans): 3-2                                                             --
│    │    │    └─AmplitudeToDB (amplitude_to_DB): 4-1                                           --
│    │    │    └─MelSpectrogram (MelSpectrogram): 4-2                                           --
│    │    │    │    └─Spectrogram (spectrogram): 5-1                                            --
│    │    │    │    └─MelScale (mel_scale): 5-2                                                 --
│    

### utils for testing

In [6]:
from s3prl.adapters.layer import AdapterLayer, AdapterLayerBase
from s3prl.adapters.configuration import AdapterConfig
for (i, layer) in model.transformer.model.iter_layers():
    for module in layer.modules():
        if (isinstance(module, AdapterLayerBase)):
            print(module.config.adapters.match(
                "acu",
                config_type=AdapterConfig,
                layer_idx=i,
                location_key="self",
            ))
            print(module.config.adapters.to_dict())
    break

None
{'adapters': {}, 'config_map': {}, 'fusions': {}, 'fusion_config_map': {}}
None
{'adapters': {}, 'config_map': {}, 'fusions': {}, 'fusion_config_map': {}}
None
{'adapters': {}, 'config_map': {}, 'fusions': {}, 'fusion_config_map': {}}


In [3]:
model.transformer.model.config.hahaha = "hahaha"
model.transformer.model.config

TransformerConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "attention_probs_dropout_prob": 0.1,
  "hahaha": "hahaha",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "transformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pre_layer_norm": false,
  "share_layer": false,
  "transformers_version": "4.17.0"
}

In [6]:
model.transformer.model.encoder.layer[0].output.config

TransformerConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "attention_probs_dropout_prob": 0.1,
  "hahaha": "hahaha",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "transformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pre_layer_norm": false,
  "share_layer": false,
  "transformers_version": "4.17.0"
}

In [5]:
model.transformer.model.base_model.encoder.layer[0].output.config

TransformerConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "transformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pre_layer_norm": false,
  "share_layer": false,
  "transformers_version": "4.17.0"
}

In [5]:
model.transformer.model.add_adapter("acu")
print(model.transformer.model.config.adapters.to_dict())
print(model)

2022-05-24 19:53:17 | INFO | s3prl.adapters.configuration | Adding adapter 'acu'.


{'adapters': {'acu': 'pfeiffer'}, 'config_map': {}, 'fusions': {}, 'fusion_config_map': {}}
UpstreamExpert(
  (transformer): PretrainedTransformer(
    (extracter): OnlinePreprocessor(
      (_melscale): MelScale()
      (_mfcc_trans): MFCC(
        (amplitude_to_DB): AmplitudeToDB()
        (MelSpectrogram): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
      )
    )
    (model): TransformerModel(
      (shared_parameters): ModuleDict()
      (invertible_adapters): ModuleDict()
      (input_representations): TransformerInputRepresentations(
        (spec_transform): Linear(in_features=80, out_features=768, bias=True)
        (LayerNorm): TransformerLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): TransformerEncoder(
        (layer): ModuleList(
          (0): TransformerLayer(
            (attention): TransformerAttention(
              (self): TransformerSelfAttention(
                (query): L